# Data Cleaning with DeepSeek Local API

This notebook demonstrates how to use the DeepSeek model via Ollama's local API for data cleaning tasks.

In [ ]:
# Install required packages
%pip install pandas numpy requests jupyter ipykernel --quiet
%pip list

In [1]:
# Verify installations
import re
import pkg_resources
required = {'pandas': '2.0.0', 'numpy': '1.24.0', 'requests': '2.31.0'}

for package, min_version in required.items():
    version = pkg_resources.get_distribution(package).version
    print(f"{package}: {version} (required: >={min_version})")

pandas: 2.2.3 (required: >=2.0.0)
numpy: 2.2.2 (required: >=1.24.0)
requests: 2.32.3 (required: >=2.31.0)


/var/folders/4d/kqcyj_q56yv6d2_qxqz3hck00000gn/T/ipykernel_68033/2948547.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime, timedelta

# Ollama API endpoint
OLLAMA_API = "http://localhost:11434/api/generate"

In [3]:
def ask_deepseek(prompt, model="deepseek-r1:1.5b"):
    """Send a prompt to DeepSeek via Ollama API"""
    response = requests.post(OLLAMA_API, json={
        "model": model,
        "prompt": prompt,
        "stream": False
    })
    return response.json().get('response', '')

In [5]:
# Generate more realistic sample data
np.random.seed(42)

# Helper function to generate messy names
def generate_messy_name():
    first_names = ['John', 'Jane', 'Robert', 'Maria', 'David', 'Sarah', 'Michael', 'Lisa', 'James', 'Emily']
    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Rodriguez', 'Martinez']
    
    first = np.random.choice(first_names)
    last = np.random.choice(last_names)
    
    # Add random spaces and occasional casing issues
    if np.random.random() < 0.3:
        first = first + ' ' * np.random.randint(1, 4)
    if np.random.random() < 0.2:
        last = last.lower()
    return f"{first} {last}"

# Helper function to generate messy emails
def generate_messy_email(name):
    domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'outlook.com', 'example.com']
    name = name.lower().replace(' ', '.')
    domain = np.random.choice(domains)
    email = f"{name}@{domain}"
    # Add common email typos
    if np.random.random() < 0.2:
        email = email.replace('@', '@@')
    if np.random.random() < 0.1:
        email = email.replace('.com', '..com')
    return email

# Generate sample data
n_samples = 10  # ลดจาก 50 เป็น 10
names = [generate_messy_name() for _ in range(n_samples)]
emails = [generate_messy_email(name.split()[0]) for name in names]

# Generate ages with typical data issues
ages = []
for _ in range(n_samples):
    if np.random.random() < 0.1:
        ages.append(None)  # Missing values
    elif np.random.random() < 0.1:
        ages.append('N/A')  # Text instead of number
    elif np.random.random() < 0.1:
        ages.append(str(np.random.randint(20, 70)) + 'y')  # Numbers with text
    elif np.random.random() < 0.1:
        ages.append('O' + str(np.random.randint(2, 6)))  # Common OCR errors
    else:
        ages.append(str(np.random.randint(20, 70)))

# Generate dates with issues
dates = []
base_date = datetime(2023, 1, 1)
for _ in range(n_samples):
    days = np.random.randint(-365, 365)
    date = base_date + timedelta(days=days)
    if np.random.random() < 0.2:
        formats = ['%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y', '%Y/%m/%d']
        dates.append(date.strftime(np.random.choice(formats)))
    elif np.random.random() < 0.1:
        dates.append('Invalid Date')
    else:
        dates.append(date.strftime('%Y-%m-%d'))

# Generate salary data with issues
salaries = []
for _ in range(n_samples):
    base_salary = np.random.randint(30000, 120000)
    if np.random.random() < 0.15:
        salaries.append(f"${base_salary:,}")  # With currency symbol
    elif np.random.random() < 0.1:
        salaries.append(f"{base_salary:,}USD")  # With currency code
    elif np.random.random() < 0.1:
        salaries.append(None)  # Missing values
    else:
        salaries.append(str(base_salary))

# Create DataFrame with messy data
df = pd.DataFrame({
    'name': names,
    'email': emails,
    'age': ages,
    'date_joined': dates,
    'salary': salaries
})

# Add some completely missing rows (ลดจำนวน missing rows ลงด้วย)
df.loc[np.random.choice(df.index, 1), :] = None  # เปลี่ยนจาก 3 เป็น 1

print("Sample of original messy data:")
display(df.head(10))
print("\nDataset shape:", df.shape)
print("\nMissing values summary:")
display(df.isnull().sum())

Sample of original messy data:


name                email   age date_joined   salary
0   Michael Brown  michael@outlook.com    O2  2022-08-29    85016
1    David miller   david@outlook..com   30y  2023-04-30    33748
2      Lisa Jones       lisa@gmail.com    61  2022-06-20   103530
3     Sarah Jones      sarah@yahoo.com    53  2023-11-30  $84,384
4  Sarah  Johnson   sarah@@outlook.com  None  2023-11-13    69353
5   Emily  Garcia      emily@yahoo.com    41  2023-10-08  $53,664
6  Emily Williams    emily@outlook.com    20  2022-12-06     None
7            None                 None  None        None     None
8   James  Miller    james@example.com   N/A  2022-08-19    80859
9   Jane martinez       jane@yahoo.com   N/A  2022-10-10    82662


Dataset shape: (10, 5)

Missing values summary:


name           1
email          1
age            2
date_joined    1
salary         2
dtype: int64

In [ ]:
# Example: Ask DeepSeek for data cleaning suggestions
sample_data = df.head(10).to_string()
prompt = f"""Given this dataset with common data quality issues:

{sample_data}

Suggest Python code to clean this data by:
1. Fixing multiple spaces and case issues in names
2. Validating and correcting email formats
3. Converting ages to numeric values (remove text, fix OCR errors)
4. Standardizing dates to YYYY-MM-DD format
5. Cleaning salary values to numeric (remove currency symbols and commas)
Only provide the Python code without explanations."""

cleaning_code = ask_deepseek(prompt)
print("Suggested cleaning code:")
print(cleaning_code)

In [ ]:
# Execute the cleaning suggestions (after reviewing the code)
# Note: Always review AI-generated code before execution

# Example manual cleaning:
df['name'] = df['name'].str.replace(r'\s+', ' ').str.strip()
df['email'] = df['email'].str.replace(r'@+', '@').str.replace(r'\.+', '.')
df['age'] = pd.to_numeric(df['age'].replace({'3O': '30', 'NA': None}), errors='coerce')

print("\nCleaned data:")
display(df)

## Advanced Data Cleaning Functions

You can create more specific cleaning functions based on DeepSeek's suggestions

In [6]:
def extract_python_code(response):
    """Extract only Python code from the response"""
    if '```python' in response:
        # Get code between ```python and ```
        code_blocks = response.split('```python')
        for block in code_blocks[1:]:
            if '```' in block:
                code = block.split('```')[0].strip()
                return code
    return None

def get_cleaning_strategy(column_data, column_name):
    """Get cleaning strategy from DeepSeek for specific column"""
    sample = column_data.head().to_string()
    prompt = f"""Given this {column_name} column data:
{sample}

Write a Python code that cleans this {column_name} column.
Return only the code without any explanation or markdown formatting.
Do not include the column name in variable names."""
    
    response = ask_deepseek(prompt)
    return extract_python_code(response)

In [7]:
# Get specific cleaning strategies for each column
columns_to_clean = ['name', 'email', 'age', 'date_joined', 'salary']

print("Requesting cleaning strategies for each column...\n")
for column in columns_to_clean:
    print(f"\n=== Cleaning strategy for {column} column ===")
    cleaning_code = get_cleaning_strategy(df[column], column)
    
    if cleaning_code:
        print("\nSuggested cleaning code:")
        print(cleaning_code)
        try:
            # Create a safe local scope for execution
            local_vars = {'df': df.copy(), 'pd': pd, 're': re, 'np': np}
            
            # Execute the code in the local scope
            exec(cleaning_code, globals(), local_vars)
            
            # Update the original dataframe with cleaned data
            if 'cleaned_data' in local_vars:
                df[column] = local_vars['cleaned_data']
            elif 'df' in local_vars and column in local_vars['df']:
                df[column] = local_vars['df'][column]
                
            print(f"\nCleaned {column} column. Preview:")
            display(df[[column]].head())
        except Exception as e:
            print(f"Error executing cleaning code: {str(e)}")
    else:
        print("No valid Python code found in response")

Requesting cleaning strategies for each column...


=== Cleaning strategy for name column ===
No valid Python code found in response

=== Cleaning strategy for email column ===

Suggested cleaning code:
def formatEmail(email):
    parts = email.split('@')
    domain_names = [p.strip() for p in parts if p.strip()]
    sorted_domains = sorted(domain_names)
    return '@'.join(sorted_domains)

Cleaned email column. Preview:


,email
0,michael@outlook.com
1,david@outlook..com
2,lisa@gmail.com
3,sarah@yahoo.com
4,sarah@@outlook.com



=== Cleaning strategy for age column ===

Suggested cleaning code:
age_data = {
    0: 'O2',
    1: '30y',
    2: '61',
    3: '53'
}
age_data[4] = None

print(age_data)
{0: 'O2', 1: '30y', 2: '61', 3: '53', 4: None}

Cleaned age column. Preview:


,age
0,O2
1,30y
2,61
3,53
4,None



=== Cleaning strategy for date_joined column ===

Suggested cleaning code:
import pandas as pd

# Assuming 'data' is your DataFrame
data['date_joined'] = pd.to_datetime(data['date_joined'])
Error executing cleaning code: name 'data' is not defined

=== Cleaning strategy for salary column ===

Suggested cleaning code:
# Assuming 'salaries' is the list of strings in the column
cleaned_salaries = []
for s in salaries:
    # Remove leading and trailing whitespace
    s_clean = s.strip()
    # Remove dollar sign if present
    if '$' in s_clean:
        s_clean = s_clean.replace('$', '')
    # Convert to integer
    cleaned_salaries.append(int(s_clean))
Error executing cleaning code: invalid literal for int() with base 10: '84,384'


In [8]:
# Show final cleaned dataset
print("Final cleaned dataset:")
display(df)

# Show data quality improvements
print("\nMissing values after cleaning:")
display(df.isnull().sum())

print("\nData types after cleaning:")
display(df.dtypes)

Final cleaned dataset:


name                email   age date_joined   salary
0   Michael Brown  michael@outlook.com    O2  2022-08-29    85016
1    David miller   david@outlook..com   30y  2023-04-30    33748
2      Lisa Jones       lisa@gmail.com    61  2022-06-20   103530
3     Sarah Jones      sarah@yahoo.com    53  2023-11-30  $84,384
4  Sarah  Johnson   sarah@@outlook.com  None  2023-11-13    69353
5   Emily  Garcia      emily@yahoo.com    41  2023-10-08  $53,664
6  Emily Williams    emily@outlook.com    20  2022-12-06     None
7            None                 None  None        None     None
8   James  Miller    james@example.com   N/A  2022-08-19    80859
9   Jane martinez       jane@yahoo.com   N/A  2022-10-10    82662


Missing values after cleaning:


name           1
email          1
age            2
date_joined    1
salary         2
dtype: int64


Data types after cleaning:


name           object
email          object
age            object
date_joined    object
salary         object
dtype: object